In [ ]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from bs4 import BeautifulSoup
from tqdm import tqdm

def croll_page(driver,html):
    soup = BeautifulSoup(html, 'html.parser')
    store_names = [store.text.strip() for store in soup.select('.head_item > .tit_name > .link_name')]

    for name in store_names:
        print(name)

    # 가게 상세 페이지 크롤링
    #store_names = store_names[:1]
    print(store_names)
    for store_name in tqdm(store_names):
        try:
            name_element = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.LINK_TEXT, store_name)))
            ActionChains(driver).move_to_element(name_element).click().perform()
            time.sleep(1)
            
            # 상세 페이지 링크 가져오기
            #place_links = WebDriverWait(driver, 5).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'a[data-id="name"]')))
            place_links = driver.find_elements(By.CLASS_NAME, 'moreview')
            for link in place_links:
                href = link.get_attribute("href")
                if href and "none" not in href:
                    print(f"{store_name} 링크: {href}")
                    driver.execute_script(f"window.open('{href}');")
                    time.sleep(1)
                    driver.switch_to.window(driver.window_handles[-1])
                    time.sleep(1)

                    # 상세 페이지 크롤링
                    html = driver.page_source
                    soup = BeautifulSoup(html, 'html.parser')

                    st_name = soup.find('h3', class_='tit_place')
                    if st_name :
                        st_name = st_name.get_text(strip=True).replace('장소명', '')
                    else:
                        st_name = '이름 없음'
                    
                    rating = soup.find('span', class_='num_star')
                    if rating :
                        rating = rating.get_text(strip=True).replace('장소명', '')
                    else:
                        rating = '평점 없음'
                    
                    reviews = soup.find('a', class_='link_review')

                    if reviews :
                        reviews = reviews.get_text(strip=True).replace('장소명', '')
                    else:
                        reviews = '이름 없음'
                    
                    blog_reviews = soup.find('a', class_='link_blog')

                    if blog_reviews :
                        blog_reviews = blog_reviews.get_text(strip=True).replace('장소명', '')
                    else:
                        blog_reviews = '이름 없음'
    
                    # 장소 카테고리 크롤링
                    category_element = soup.find('span', class_='info_cate')

                    if category_element :
                        category = category_element.get_text(strip=True).replace('장소명', '')
                    else:
                        category = '이름 없음'
                    
                    
                    # 메뉴 탭 클릭 (다양한 구조 대응)
                    try:
                        menu_tab = WebDriverWait(driver, 5).until(
                            EC.element_to_be_clickable((By.CSS_SELECTOR, "li:nth-child(3) > a.link_tab"))
                        )
                    except:
                        menu_tab = driver.find_element(By.CSS_SELECTOR, "a.link_tab[role='tab']")
                    
                    if menu_tab:
                        menu_tab.click()
                        time.sleep(1)
                    else:
                        print(f"{store_name} 메뉴 탭 없음")

                    # 메뉴 정보 크롤링
                    menu_items = soup.find_all('div', class_='info_goods')
                    
                    if menu_items:
                        menu_count = len(menu_items)
                        if menu_count >= 3:
                            menu_items = menu_items[:3]  # 메뉴가 3개 이상이면 상위 3개만 가져옴
                        menus=[]
                        for item in menu_items:
                            if item.find('strong', class_='tit_item') and item.find('p', class_='desc_item') :
                                menus.append([item.find('strong', class_='tit_item').get_text(strip=True), item.find('p', class_='desc_item').get_text(strip=True)])
                            else:
                                pass
                    else:
                        menus = []
                        

                    print(f'장소명: {st_name}')
                    print(f'별점: {rating}')
                    print(f'후기 개수: {reviews}')
                    print(f'블로그 후기 개수: {blog_reviews}')
                    print(f'카테고리: {category}')
                    print('메뉴 목록:')
                    if menus:
                        for menu in menus:
                            print(f'- {menu[0]}: {menu[1]}')
                    else:
                        print('메뉴 없음')

                    # 창 닫고 원래 창으로 복귀
                    driver.close()
                    time.sleep(1)
                    driver.switch_to.window(driver.window_handles[0])
                    break
        except Exception as e:
            print(f"{store_name} 클릭 실패: {e}")

# 카카오맵 URL
URL = 'https://map.kakao.com/'
KEYWORD = '가산동 맛집'

# Chrome WebDriver 설정
options = webdriver.ChromeOptions()
options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3')
options.add_argument('window-size=1380,900')

driver = webdriver.Chrome(options=options)
driver.get(URL)

# 검색어 입력 및 검색 실행
search_area = driver.find_element(By.XPATH, '//*[@id="search.keyword.query"]')
search_area.send_keys(KEYWORD)
driver.find_element(By.XPATH, '//*[@id="search.keyword.submit"]').send_keys(Keys.ENTER)
time.sleep(1)

# 장소 탭 클릭
driver.find_element(By.XPATH, '//*[@id="info.main.options"]/li[2]/a').send_keys(Keys.ENTER)
time.sleep(1)

#페이지별로 상점별 데이터 수집
j=0
while True:
    num_pages = 5
    #5개의 페이지 순환
    for i in range(num_pages):
        print("page"+str((i+1)+5*j))
        #페이지 번호 선정
        page_section = driver.find_element(By.ID,'info.search.page')
        page_elements = page_section.find_elements(By.TAG_NAME,'a')
        page = page_elements[i]
        #페이지 클릭
        driver.execute_script("arguments[0].click();", page)
        time.sleep(1)
        html = driver.page_source
        #상점별 데이터 크롤링
        croll_page(driver,html)
    #그 후 다음 버튼 찾기
    next_button = driver.find_element(By.ID,'info.search.page.next')

        #다음 버튼 없는경우 끝
    if 'disabled' in next_button.get_attribute('class'):
        print('다음 페이지 없음.종료')
        break
    else:
        #다음 버튼 있는경우 다음 버튼 클릭
        j=j+1
        next_button.click()
        time.sleep(1)


# 드라이버 종료
driver.quit()

page1
춘천옥
삼숙이갈비 본점
가산어시장
스시쥬베이
북촌삼대갈비 가산본점
도원
인크커피 가산점
개성손만두요리전문점 가산디지털점
동산미
가산물갈비&즉석소불고기
맥도날드 가산비지니스센터점
호세가
풍년갈비
동남집
통영해물가 가산2호점
['춘천옥', '삼숙이갈비 본점', '가산어시장', '스시쥬베이', '북촌삼대갈비 가산본점', '도원', '인크커피 가산점', '개성손만두요리전문점 가산디지털점', '동산미', '가산물갈비&즉석소불고기', '맥도날드 가산비지니스센터점', '호세가', '풍년갈비', '동남집', '통영해물가 가산2호점']


  0%|          | 0/15 [00:00<?, ?it/s]

춘천옥 링크: https://place.map.kakao.com/8117410
장소명: 춘천옥
별점: 2.8
후기 개수: 후기53개
블로그 후기 개수: 블로그124개
카테고리: 장소 카테고리족발,보쌈
메뉴 목록:
- 김치가 맛있는 건강한 보쌈: 36,000원
- 메밀 막국수 （수제면）: 10,000원
- 선지국밥: 10,000원


  7%|▋         | 1/15 [00:05<01:19,  5.66s/it]

삼숙이갈비 본점 링크: https://place.map.kakao.com/8117410
장소명: 춘천옥
별점: 2.8
후기 개수: 후기53개
블로그 후기 개수: 블로그124개
카테고리: 장소 카테고리족발,보쌈
메뉴 목록:
- 김치가 맛있는 건강한 보쌈: 36,000원
- 메밀 막국수 （수제면）: 10,000원
- 선지국밥: 10,000원


...

도원 링크: https://place.map.kakao.com/8117410
장소명: 춘천옥
별점: 2.8
후기 개수: 후기53개
블로그 후기 개수: 블로그124개
카테고리: 장소 카테고리족발,보쌈
메뉴 목록:
- 김치가 맛있는 건강한 보쌈: 36,000원
- 메밀 막국수 （수제면）: 10,000원
- 선지국밥: 10,000원


 40%|████      | 6/15 [00:33<00:50,  5.64s/it]

인크커피 가산점 링크: https://place.map.kakao.com/8117410
장소명: 춘천옥
별점: 2.8
후기 개수: 후기53개
블로그 후기 개수: 블로그124개
카테고리: 장소 카테고리족발,보쌈
메뉴 목록:
- 김치가 맛있는 건강한 보쌈: 36,000원
- 메밀 막국수 （수제면）: 10,000원
- 선지국밥: 10,000원


 47%|████▋     | 7/15 [00:40<00:45,  5.72s/it]


KeyboardInterrupt: 